In [2]:
# run in a notebook cell
%pip install statsmodels

  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 25.8 MB/s eta 0:00:00a 0:00:01
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# run in a notebook cell so it installs into the current kernel
%pip install prophet

  Using cached cmdstanpy-1.2.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached stanio-0.5.1-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 36.7 MB/s eta 0:00:00 0:00:01
Using cached cmdstanpy-1.2.5-py3-none-any.whl (94 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
Using cached stanio-0.5.1-py3-none-any.whl (8.1 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.fft import fft
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from dotenv import load_dotenv
# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "transformed_features_and_target_top50.parquet")
df

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,temp_t-1,wind_speed_t-3,wind_speed_t-2,wind_speed_t-1,precipitation_t-3,precipitation_t-2,precipitation_t-1,target,start_station_id,hour
0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,A32002,2024-10-28 22:00:00
1,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,A32002,2024-10-28 23:00:00
2,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,A32002,2024-10-29 00:00:00
3,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,A32002,2024-10-29 01:00:00
4,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,8.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,A32002,2024-10-29 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404579,16.0,9.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,M32085,2025-09-30 19:00:00
404580,9.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,M32085,2025-09-30 20:00:00
404581,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,M32085,2025-09-30 21:00:00
404582,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,M32085,2025-09-30 22:00:00


In [3]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2025, 7, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(294163, 683)
(294163,)
(110421, 683)
(110421,)


In [4]:
def add_fft_features(X):
    fft_features = []
    for col in X.columns:
        if col.startswith('rides_'):
            fft_result = fft(X[col])
            fft_features.append(pd.Series(np.abs(fft_result), name=f'fft_{col}'))
    return pd.concat([X] + fft_features, axis=1)

X_train_fft = add_fft_features(X_train)
X_test_fft = add_fft_features(X_test)

In [5]:
def fit_arima(train, test, order=(5,1,0)): #order=(1,1,1)
    # Fit ARIMA model
    model = ARIMA(train, order=order)
    results = model.fit()

    # Generate forecasts
    forecast = results.forecast(steps=len(test))
    return forecast

In [ ]:
arima_forecast = fit_arima(y_train, y_test)
arima_mae = mean_absolute_error(y_test, arima_forecast)
print(f"ARIMA MAE: {arima_mae:.4f}")

ARIMA MAE: 4.3631


: 